# Kaggle COVID19 Global Forecast

https://www.kaggle.com/c/covid19-global-forecasting-week-2

In [1]:
%load_ext autoreload
%autoreload 2
%autosave 0

from pathlib import Path
from pandas import DataFrame, Series, date_range, read_csv

FIGSIZE = (9, 5)
FOLDER = Path("../data/kaggle/global/raw")

Autosave disabled


## get data
Load training data and prediction dates.

In [2]:
def load(path, **kwargs):
    """ DataFrame: Load Kaggle data. """
    kwset = kwargs.setdefault
    kwset('parse_dates',  ['Date'])
    kwset('index_col', 'Country_Region Province_State Date'.split())
    
    return read_csv(path, **kwargs)
    

train = load(FOLDER / 'train.csv')
train.head()

Id  ConfirmedCases  Fatalities
Country_Region Province_State Date                                      
Afghanistan    NaN            2020-01-22   1             0.0         0.0
                              2020-01-23   2             0.0         0.0
                              2020-01-24   3             0.0         0.0
                              2020-01-25   4             0.0         0.0
                              2020-01-26   5             0.0         0.0

In [3]:
local = train.xs('China').xs('Hubei')
local.head()

,Id,ConfirmedCases,Fatalities
Date,,,
2020-01-22,6201,444.0,17.0
2020-01-23,6202,444.0,17.0
2020-01-24,6203,549.0,24.0
2020-01-25,6204,761.0,40.0
2020-01-26,6205,1058.0,52.0


In [4]:
test = load(FOLDER / 'test.csv')
test.head()

ForecastId
Country_Region Province_State Date                  
Afghanistan    NaN            2020-03-19           1
                              2020-03-20           2
                              2020-03-21           3
                              2020-03-22           4
                              2020-03-23           5

## model
Create a [contagion window model](https://github.com/samkennerly/covid19).

In [ ]:
def convolved(deltas, *window):
    return sum( w * x for w, x in zip(window, reversed(deltas)) )

def new_cases(deltas, *window, n=POPULATION, vax_rate=0):
    return (1 - vax_rate - sum(deltas) / n ) * convolved(deltas, *window)

def predict(steps, deltas, window, n=POPULATION, mu=MORTALITY, vax_rate=0):
    tau = len(window)
    
    

    delayed = deltas[-tau] if (len(deltas) > tau) else 0
    unexposed = n - sum(deltas)
    
    deceased = int(mu * delayed)
    recovered = delayed - deceased
    
    infectious = convolved(deltas, *( 1 if w else 0 for w in window ))
    quarantined = convolved(deltas, *( 0 if w else 1 for w in window ))
    
    vaccinated = int(vax_rate * unexposed)
    susceptible = unexposed - vaccinated

    return deceased, recovered, infectious, quarantined, susceptible, vaccinated
    
tomorrow(deltas[:20], window)